### Creating dimension table

In [0]:
CREATE TABLE parking_catalog.parking_schema.dim_rate (
    pole STRING,
    hourly_rate INT,
    rate_type STRING,
    start_date DATE,
    end_date DATE,
    is_current BOOLEAN
)
USING DELTA;

In [0]:
-- inserting old data
INSERT INTO parking_catalog.parking_schema.dim_rate
SELECT
  pole,
  hourly_rate,
  rate_type,
  current_date() AS start_date,
  NULL AS end_date,
  true AS is_current
FROM parking_catalog.parking_schema.rate_table;

num_affected_rows,num_inserted_rows
4654,4654


In [0]:
-- source data
CREATE TABLE parking_catalog.parking_schema.rate_updates (
    pole STRING,
    hourly_rate INT,
    rate_type STRING
)
USING DELTA;

In [0]:
Insert into parking_catalog.parking_schema.rate_updates
values ('CC-1003',4,'weekday'),
       ('CC-2001', 5, 'weekday'),
       ('CC-1015',8,'weekday');

num_affected_rows,num_inserted_rows
3,3


## SCD Implemenation through MERGE INTO

In [0]:
MERGE INTO parking_catalog.parking_schema.dim_rate AS target
USING parking_catalog.parking_schema.rate_updates AS source
ON target.pole = source.pole
AND target.is_current = true

WHEN MATCHED AND target.hourly_rate <> source.hourly_rate THEN
  UPDATE SET
    target.end_date = current_date(),
    target.is_current = false

WHEN NOT MATCHED THEN
  INSERT (
    pole,
    hourly_rate,
    rate_type,
    start_date,
    end_date,
    is_current
  )
  VALUES (
    source.pole,
    source.hourly_rate,
    source.rate_type,
    current_date(),
    NULL,
    true
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1
